In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

21/10/20 23:50:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
sdf = spark.read.options(header=True, inferSchema=True).csv('data/kddcup99.csv')
sdf.limit(10).toPandas()

21/10/20 23:51:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.0,0.0,0.0,0.0,b'normal.'
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.0,0.0,0.0,0.0,b'normal.'
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.0,0.0,0.0,0.0,b'normal.'
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.0,0.0,0.0,0.0,b'normal.'
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.0,0.0,0.0,0.0,b'normal.'
5,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,59,1.0,0.0,0.02,0.00,0.0,0.0,0.0,0.0,b'normal.'
6,0,b'tcp',b'http',b'SF',212,1940,0,0,0,0,...,69,1.0,0.0,1.00,0.04,0.0,0.0,0.0,0.0,b'normal.'
7,0,b'tcp',b'http',b'SF',159,4087,0,0,0,0,...,79,1.0,0.0,0.09,0.04,0.0,0.0,0.0,0.0,b'normal.'
8,0,b'tcp',b'http',b'SF',210,151,0,0,0,0,...,89,1.0,0.0,0.12,0.04,0.0,0.0,0.0,0.0,b'normal.'
9,0,b'tcp',b'http',b'SF',212,786,0,0,0,1,...,99,1.0,0.0,0.12,0.05,0.0,0.0,0.0,0.0,b'normal.'


In [3]:
from faas.data import get_non_numeric_columns

non_numeric_columns = get_non_numeric_columns(sdf)
non_numeric_columns

['protocol_type', 'service', 'flag', 'labels']

User interaction
- scaling group column (optional, unique)
- target columns (required, unique)
- categorical columns (may be required, user to verify)

In [4]:
target_scaling_group_by_column = 'protocol_type'
target_column = 'dst_host_srv_count'
categorical_columns = non_numeric_columns
covariate_columns = [c for c in sdf.columns if c != target_column]


In [5]:
from faas.data import LGBMDataWrapper

lgbmdw = LGBMDataWrapper(
    target_column=target_column,
    covariate_columns=covariate_columns,
    categorical_columns=categorical_columns,
    target_scaling_group_by_column=target_scaling_group_by_column
)

In [6]:
lgbmdw.fit(sdf)

In [7]:
sdf_pred = lgbmdw.predict(sdf)

In [8]:
yactual = sdf.select(target_column).toPandas()
ypred = sdf_pred.select(target_column).toPandas()

In [9]:
yactual.head()

,dst_host_srv_count
0,9
1,19
2,29
3,39
4,49


In [10]:
ypred.head()

,dst_host_srv_count
0,29.079795
1,20.015811
2,39.026580
3,41.361745
4,53.008258
